In [1]:
# Imports
from __future__ import print_function
import SimpleITK as sitk
import sys, time, os, numpy as np

In [2]:
# Load the numpy array and swap axis needed
arr = np.load('001.npy')
narr = np.swapaxes(arr,0,2)  ######## MOST IMPORTANT LINE, SEE WHAT AXIS TO SWAP TO HAVE THE DICOM IN THE RIGHT ORDER

# Modify the image (blurring)
filtered_image = sitk.GetImageFromArray(narr)
series_file_names = [f'output/DCM_{i}.dcm' for i in range(arr.shape[2])]

# Convert numpy array to int
castFilter = sitk.CastImageFilter()
castFilter.SetOutputPixelType(sitk.sitkInt16)
filtered_image = castFilter.Execute(filtered_image)

# Show the depth, should be 41!!
print(filtered_image.GetDepth())

41


In [3]:
# Writes all the image to output/DCM_i.dcm
writer = sitk.ImageFileWriter()
writer.KeepOriginalImageUIDOn()
modification_time = time.strftime("%H%M%S")
modification_date = time.strftime("%Y%m%d")

for i in range(filtered_image.GetDepth()): 
    image_slice = filtered_image[:,:,i]
    
    # Set relevant keys indicating the change, modify or remove private tags as needed
    image_slice.SetMetaData("0008|0031", modification_time)
    image_slice.SetMetaData("0008|0021", modification_date)
    image_slice.SetMetaData("0008|0008", "DERIVED\SECONDARY")
    
    # Each of the UID components is a number (cannot start with zero) and separated by a '.'
    # We create a unique series ID using the date and time.
    image_slice.SetMetaData("0020|000e", "1.2.826.0.1.3680043.2.1125."+modification_date+".1"+modification_time)
    
    # Write to the output directory and add the extension dcm if not there, to force writing is in DICOM format.
    writer.SetFileName( series_file_names[i] )
    writer.Execute(image_slice)